Momenta of pertTrident
=================

Here we prototype the momenta of pertTrident in array evaluation


In [1]:
import numpy as np
import perttrident.qft as qft

Kinematic Parameters
---------

In [22]:
def absP(en,m=1):
    return np.sqrt(en**2 - 1)

def buildPara(ss,e1,cth1,phi1,e2,cth2):
    SS = ss[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
    E1 = e1[np.newaxis,:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
    C1 = cth1[np.newaxis,np.newaxis,:,np.newaxis,np.newaxis,np.newaxis]
    PH1 = phi1[np.newaxis,np.newaxis,np.newaxis,:,np.newaxis,np.newaxis]
    E2 = e2[np.newaxis,np.newaxis,np.newaxis,np.newaxis,:,np.newaxis]
    C2 = cth2[np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,:]
    return [SS,E1,C1,PH1,E2,C2]

ss = np.array([3.353])
E1 = np.linspace(1.1,3.5,100)
cTh1 = np.array([0.965])
phi1 = np.array([0.0])
E2 = np.linspace(1.1,3.5,20)
cTh2 = np.linspace(0.9,0.99,11)#np.array([0.95,0.965])

kinGrid = buildPara(ss,E1,cTh1,phi1,E2,cTh2)



In [23]:
omega = (kinGrid[0]**2 - 1)/2.0
E= np.ones(kinGrid[0].shape)
rho1 = np.sqrt(kinGrid[1]**2 - 1)
sTh1 = np.sin(np.arccos(kinGrid[2]))
rho2 = np.sqrt(kinGrid[1]**2 - 1)
sTh2 = np.sin(np.arccos(kinGrid[2]))





Bound of $\cos\theta_1$:
--------

In [24]:
# bound of cThetaA
def a1(Et,pt,Ea,Eb,cThetab):
    s=Et**2 - pt**2
    return s+1.0 - 2.0*(Et*Eb - pt*absP(Eb)*cThetab + Ea*Et - Ea*Eb)

def b1(Et,pt,Ea,Eb,cThetab):
    return 2.0*(absP(Ea)*pt - absP(Ea)*absP(Eb)*cThetab)

def c1(Et,pt,Ea,Eb,cThetab):
    return 2.0*absP(Ea)*absP(Eb)*np.sqrt(1-cThetab**2)


def a2(Et,pt,Ea,Eb,cThetab):
    return b1(Et,pt,Ea,Eb,cThetab)**2 + c1(Et,pt,Ea,Eb,cThetab)**2

def b2(Et,pt,Ea,Eb,cThetab):
    return 2.0 * a1(Et,pt,Ea,Eb,cThetab)*b1(Et,pt,Ea,Eb,cThetab)

def c2(Et,pt,Ea,Eb,cThetab):
    return a1(Et,pt,Ea,Eb,cThetab)**2 - c1(Et,pt,Ea,Eb,cThetab)**2

def discrimCthetaA(Et,pt,Ea,Eb,cThetab):
    return b2(Et,pt,Ea,Eb,cThetab)**2 - 4.0*a2(Et,pt,Ea,Eb,cThetab)*c2(Et,pt,Ea,Eb,cThetab)



Old fashion way:
-----

In [25]:

def upCthetaA(Et,pt,Ea,Eb,cThetab):
    discr = discrimCthetaA(Et,pt,Ea,Eb,cThetab)
    if discr>=0:
        temp = (-b2(Et,pt,Ea,Eb,cThetab) + np.sqrt(discr))/(2.0*a2(Et,pt,Ea,Eb,cThetab))
        return temp
    else:
        return False

def lowCthetaA(Et,pt,Ea,Eb,cThetab):
    discr = discrimCthetaA(Et,pt,Ea,Eb,cThetab)
    if discr>=0:
        return (-b2(Et,pt,Ea,Eb,cThetab) - np.sqrt(discr))/(2.0*a2(Et,pt,Ea,Eb,cThetab))
    else:
        return False

def charCthetaA(Et,pt,Ea,Eb,cThA,cThetab):
    tempUp = upCthetaA(Et,pt,Ea,Eb,cThetab)
    tempLow = lowCthetaA(Et,pt,Ea,Eb,cThetab)
    if tempUp and tempLow:
        return (cThA < tempUp) and (cThA > tempLow)
    else:
        return False

New fashion way:
--------

In [26]:
def legitDiscr(Et,pt,Ea,Eb,cThetab):
    d = discrimCthetaA(Et,pt,Ea,Eb,cThetab)
    return d

In [29]:
Et = omega + E
pt = omega


def charCTH1(Et,pt,Ea,Eb,cThetab,cthA):
    discr = discrimCthetaA(Et,pt,Ea,Eb,cThetab)
    cond0 = discr>=0
    discr2 = discr*cond0
    res1 = ((-b2(Et,pt,Ea,Eb,cThetab) - np.sqrt(discr2))/(2.0*a2(Et,pt,Ea,Eb,cThetab)))*cond0
    cond1 = (res1 - cthA)<=0
    res2 = ((-b2(Et,pt,Ea,Eb,cThetab) + np.sqrt(discr2))/(2.0*a2(Et,pt,Ea,Eb,cThetab)))*cond0
    cond2 = (res2 - cthA)>=0
    return cond1*cond2*cond0
    
res = charCTH1(Et,pt,kinGrid[1],kinGrid[4],kinGrid[5],kinGrid[2])


Compare both:
----------

In [30]:
import time
startNEW = time.time()
res = charCTH1(Et,pt,kinGrid[1],kinGrid[4],kinGrid[5],kinGrid[2])
timeNEW = time.time() - startNEW
evals = 0
timeOLD = 0.0
for i1,e1 in enumerate(E1):
    for i2,e2 in enumerate(E2):
        for i3,c in enumerate(cTh2):
            dirRes = 0.0
            arrRes = 0.0
            startOLD = time.time()
            resOLD = charCthetaA(Et,pt,e1,e2,cTh1[0],c)
            endOLD = time.time() - startOLD
            timeOLD += endOLD
            if resOLD:
                dirRes = 1.0
            if res[0,i1,0,0,i2,i3]:
                arrRes = 1.0

            if dirRes!=arrRes:
                print "-"*20
                print "dir E1: %s"%e1
                print "dir E2: %s"%e2
                print "dir cth2: %s"%c
                print "arr E1: %s"%kinGrid[1][0,i1,0,0,0,0]
                print "arr E2: %s"%kinGrid[4][0,0,0,0,i2,0]
                print "arr cth2: %s"%kinGrid[5][0,0,0,0,0,i3]
            evals+=1
print "Done. (%d evals)"%evals
print "time NEW: %1.4e"%(timeNEW)
print "time OLD: %1.4e"%(timeOLD)

Done. (22000 evals)
time NEW: 5.2330e-03
time OLD: 7.6710e+00


In [9]:
test = kinGrid[1] * kinGrid[4]
#print test
#print test.shape
cond = test>3
better = test*cond
res = ((better/10*cond - kinGrid[5])<0)*cond

evals = 0

for i1,e1 in enumerate(E1):
    for i2,e2 in enumerate(E2):
        for i3,c in enumerate(cTh2):
            dirRes = 0.0
            arrRes = 0.0
            if e1*e2 >3:
                #print e1*e2-c
                if (e1*e2/10-c)<0:
                    dirRes = 1.0
            if res[0,i1,0,0,i2,i3]:
                arrRes = 1.0

            if dirRes!=arrRes:
                print "-"*20
                print "dir E1: %s"%e1
                print "dir E2: %s"%e2
                print "dir cth2: %s"%c
                print "arr E1: %s"%kinGrid[1][0,i1,0,0,0,0]
                print "arr E2: %s"%kinGrid[4][0,0,0,0,i2,0]
                print "arr cth2: %s"%kinGrid[5][0,0,0,0,0,i3]
            evals+=1
print "Done. (%d evals)"%evals

Done. (1100 evals)


In [10]:
True+False

1

In [57]:
def condition(cond,alter=0.0):
    def condEvaluator(func):
        def tempFunc(x,y,z):
            select_indices = np.where(cond)
            resArr=np.ones(cond.shape)*alter
            resArr[select_indices]=func(x[select_indices[0]][0][0],y[0][select_indices[1]][0],z[0][0][select_indices[2]])
            return resArr
        return tempFunc
    return condEvaluator

a=np.linspace(-1,1,10)
b=np.linspace(1,2,5)
c=np.linspace(3,4,10)
A= a[:,np.newaxis,np.newaxis]
B= b[np.newaxis,:,np.newaxis]
C= c[np.newaxis,np.newaxis,:]
startCond = time.time()
physArea = A*B*C>=0
endCond = time.time() - startCond


startBuild = time.time()
@condition(physArea)
def func(x,y,z):
    return np.sqrt(x*y*z)
endBuild = time.time() - startBuild

#for el in dir(np):
#    print el
start = time.time()
resCond = func(A,B,C)
end = time.time() - start
print "time calcCond: %1.4e"%(endCond)
print "time build: %1.4e"%(endBuild)
print "time eval: %1.4e"%(end)
print "time full: %1.4e"%(endCond+endBuild+end)

time calcCond: 1.7118e-04
time build: 1.7905e-04
time eval: 2.8300e-04
time full: 6.3324e-04


In [73]:
def funcOLD(x,y,z):
    radi = x*y*z
    if radi>0:
        return np.sqrt(radi)
    else:
        return 0.0

startOLD = time.time()
res = np.array([funcOLD(x1,x2,x3) for x1 in a for x2 in b for x3 in c])
endOLD = time.time() - startOLD
print "time OLD: %1.4e"%(endOLD)
resOLD = res.reshape(10,5,10)
for i in np.arange(10):
    for j in np.arange(5):
        for k in np.arange(10):
            print "resCond: %s"%resCond[i][j][k]
            print "resOLD:  %s"%resOLD[i][j][k]
            print "absErr:  %s"%(np.abs(resCond[i][j][k] - resOLD[i][j][k]))

time OLD: 2.5699e-03
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resCond: 0.0
resOLD:  0.0
absErr:  0.0
resC

resCond: 0.6666666666666669
resOLD:  1.8257418583505534
absErr:  1.1590751916838866
resCond: 0.5773502691896258
resOLD:  1.7078251276599328
absErr:  1.1304748584703068
resCond: 0.5879447357921314
resOLD:  1.7391639824998362
absErr:  1.1512192467077047
resCond: 0.5983516452371672
resOLD:  1.7699480357859987
absErr:  1.1715963905488316
resCond: 0.6085806194501847
resOLD:  1.8002057495577386
absErr:  1.1916251301075538
resCond: 0.6186404847588914
resOLD:  1.8299632324445114
absErr:  1.21132274768562
resCond: 0.628539361054709
resOLD:  1.8592445034090563
absErr:  1.2307051423543474
resCond: 0.6382847385042255
resOLD:  1.888071718662936
absErr:  1.2497869801587105
resCond: 0.6478835438717002
resOLD:  1.9164653678511485
absErr:  1.2685818239794484
resCond: 0.6573421981221798
resOLD:  1.9444444444444442
absErr:  1.2871022463222643
resCond: 0.6666666666666669
resOLD:  1.9720265943665383
absErr:  1.3053599276998713
resCond: 0.5773502691896258
resOLD:  1.8257418583505534
absErr:  1.2483915891609

In [63]:
sPre = time.time()
radiPre = A*B*C
ePre = time.time() - sPre
sCond = time.time()
physArea = radiPre>=0
eCond = time.time() - sCond
sBuild = time.time()
radiPost = radiPre*physArea
eBuild = time.time() - sBuild
sEval = time.time()
res = np.sqrt(radiPost)
eEval = time.time() - sEval
print res
print resCond
print "correct result: %s"%(np.close(res,resCond))
print "time calcCond: %1.4e"%(endCond)
print "time build: %1.4e"%(endBuild)
print "time eval: %1.4e"%(end)
print "time full: %1.4e"%(endCond+endBuild+end)

[[[-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]]

 [[-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.         -0.
   -0.         -0.         -0.         -0.         -0.        ]
  [-0.         -0.         -0.         -0.    

AttributeError: 'module' object has no attribute 'close'

True

In [71]:
[[ 0.57735027  0.58794474  0.59835165  0.60858062  0.61864048
    0.62853936  0.63828474  0.64788354  0.6573422   0.66666667]
  [ 0.64549722  0.6573422   0.66897748  0.68041382  0.69166109
    0.70272837  0.71362403  0.72435582  0.73493092  0.74535599]
  [ 0.70710678  0.7200823   0.73282811  0.74535599  0.75767676
    0.76980036  0.78173596  0.79349205  0.80507649  0.81649658]
  [ 0.76376262  0.77777778  0.79154482  0.80507649  0.81838444
    0.83147942  0.84437134  0.85706937  0.86958199  0.8819171 ]
  [ 0.81649658  0.83147942  0.84619701  0.86066297  0.87488976
    0.88888889  0.90267093  0.91624569  0.92962225  0.94280904]]

[[0.57735027 0.58794474 0.59835165 0.60858062 0.61864048 0.62853936
   0.63828474 0.64788354 0.6573422  0.66666667]
  [0.64549722 0.6573422  0.66897748 0.68041382 0.69166109 0.70272837
   0.71362403 0.72435582 0.73493092 0.74535599]
  [0.70710678 0.7200823  0.73282811 0.74535599 0.75767676 0.76980036
   0.78173596 0.79349205 0.80507649 0.81649658]
  [0.76376262 0.77777778 0.79154482 0.80507649 0.81838444 0.83147942
   0.84437134 0.85706937 0.86958199 0.8819171 ]
  [0.81649658 0.83147942 0.84619701 0.86066297 0.87488976 0.88888889
   0.90267093 0.91624569 0.92962225 0.94280904]]

SyntaxError: invalid syntax (<ipython-input-71-e474765dc49f>, line 1)